TODO 
- [ ] use pytorch dists to samples and get density in order to make loss
- [ ] check equations
- [ ] train

In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))


In [2]:
import torch
from torch.nn import functional as F
from torch.autograd import Variable
from torch import nn, optim
import torch.utils.data

# load as dask array
import dask.array as da
import dask
import h5py

import os
import glob
import numpy as np
from matplotlib import pyplot as plt
from tqdm import tqdm_notebook as tqdm

/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
from vae import VAE, loss_function
from helpers.summarize import TorchSummarizeDf
from helpers.dataset import NumpyDataset, TQDMDaskProgressBar, load_npzs
from rnn import MDNRNN

In [4]:
cuda= torch.cuda.is_available()
env_name='sonic'
num_epochs=200
batch_size=64

In [5]:
# Load VAE
vae = VAE(image_size=128, z_dim=32, conv_dim=64, code_dim=16, k_dim=128)
if cuda:
    vae = vae.cuda()
# # Resume?
NAME='VAE_2xv5'
save_file = f'./models/{NAME}_state_dict.pkl'
if os.path.isfile(save_file):
    state_dict = torch.load(save_file)
    vae.load_state_dict(state_dict)
    print(f'loaded {save_file}')

loaded ./models/VAE_2xv5_state_dict.pkl


In [6]:
# # Test VAE
# # load
# data_cache_file = '/tmp/sonic_vae2.hdf5'
# data = da.from_array(h5py.File(data_cache_file, mode='r')['x'], chunks=(2000, 128, 128, 3))
# data
# data_split = int(len(data)*0.8)
# data_train = data[:data_split]
# data_test = data[data_split:]
# data_train, data_test

   
# dataset_train = NumpyDataset(data_train)
# loader_train = torch.utils.data.DataLoader(dataset_train, pin_memory=True, shuffle=False, batch_size=batch_size)


# dataset_test = NumpyDataset(data_test)
# loader_test = torch.utils.data.DataLoader(dataset_test, pin_memory=True, shuffle=False, batch_size=batch_size)

# dataset_train, loader_train

# # Plot reconstructions
# def plot_results(loader=loader_test, n=2, epoch=0):
#     x, = next(iter(loader))

#     X = Variable(x).cuda().transpose(1,3).contiguous()
#     Y, mu, logvar = vae.forward(X)
#     loss = loss_function(Y, X, mu, logvar)

#     y=Y.cpu().data.transpose(1,3).numpy()
#     for i in range(n):
#         plt.subplot(1, 2, 1)
#         plt.title('original')
#         plt.imshow(x[i].numpy())

#         plt.subplot(1, 2, 2)
#         plt.imshow(y[i])
#         plt.title('reconstructed')

#         plt.suptitle('epoch {}, index {}, loss {:2.4f}'.format(epoch, i, loss.cpu().data.numpy()))
#         plt.show()
        
# plot_results(loader=loader_test, n=2, epoch=0)

# Train

In [7]:
# take windows
# make padded sequences, we need to make the data in shape (batch, window_of_timesteps, features)

def timeseries_to_seq(x, window=3, pad=True):
    """
  Inputs:
  - x: shape (timeseries, features)
  - window: e.g. 3
  - pad: Wether to pad the input sequence or not (e.g. if you prepadded it or provided extra length)
  Outputs:
  - y: shape shape (batch, window, features)
    where batch=timeseries-window if pad=False or batch=timeseries if pad=True
    
  Usage:
  x= np.range(1000)
  x_stacked = timeseries_to_seq(x, window=50)
  m = torch.nn.LSTM()
  y =  m(x_stacked)[0]
  """
    if pad:
        x = np.pad(x, [[window, 0], [0, 0]], mode='constant')
    y = np.stack([x[i:i + window][::-1] for i in range(len(x)-window)], axis=1)
    y = np.transpose(y, (1,0,2))
    return y



In [142]:
# Plot reconstructions
def plot_results(n=2, epoch=0, batch_size=6, figsize=(6,9)):
    vae.eval()
    mdnrnn.eval()
    
    # do a rollout
    env = make_env('sonic')
    try:
        actions = []
        observations = []
        rewards=[]

        observation = env.reset()
        for i in range(batch_size):
            action = env.action_space.sample()
            observation, reward, done, env_info = env.step(action)
            actions.append(action)
            observations.append(observation)
            rewards.append(reward)
            if done:
                observation = env.reset()
    except:
        env.close()
        raise
    else:
        env.close()

    # stack
    actions = np.stack(actions)
    observations = np.stack(observations)
    rewards = np.stack(rewards)

    # Run VAE
    X = Variable(torch.from_numpy(observations)).cuda().transpose(1,3).contiguous()
    Y, mu, logvar = vae.forward(X)
    z = vae.sample(mu, logvar).data.cpu().numpy()
    loss_vae = loss_function(Y, X, mu, logvar)

    # Stack into sequences
    # TODO stack as torch vars
    actions = timeseries_to_seq(actions, window=seq_len)
    z = timeseries_to_seq(z, window=seq_len)

    # Forward
    z_v = Variable(torch.from_numpy(z)).cuda()
    actions_v = Variable(torch.from_numpy(actions.astype(np.uint8))).float().cuda()
    if cuda:
        z_v=z_v.cuda()
        actions_v=actions_v.cuda()
    z_pred, hidden_state = mdnrnn.forward(z_v, actions_v)

    loss = mdnrnn.rnn_loss(z_v, z_pred)

    y=Y.cpu().data.transpose(1,3).numpy()
    
    X_pred = vae.decode(mu)
    
    for i in np.linspace(0,batch_size-2,n):
        i=int(i)
        x_orig = X[i].transpose(0,2).data.cpu().numpy()
        x_next = X[i+1].transpose(0,2).data.cpu().numpy()
        x_pred = X_pred[i].transpose(0,2).data.cpu().numpy()
        loss_vae_i = loss_vae[i].cpu().data.item()
        loss_i = loss[i].cpu().data.item()
        
        plt.figure(figsize=figsize)
        
        plt.subplot(2, 3, 1)
        plt.axis("off")
        plt.title('original')
        plt.imshow(x_orig)

        plt.subplot(2, 3, 4)
        plt.axis("off")
        plt.imshow(y[i])
        plt.title('reconstructed')
           
        plt.subplot(2, 3, 2)
        plt.axis("off")
        plt.imshow(x_next)
        plt.title('true next')
        
        plt.subplot(2, 3, 5)
        plt.axis("off")
        plt.imshow(x_pred)
        plt.title('pred next')
        
        plt.subplot(2, 3, 3)
        plt.axis("off")
        plt.imshow(np.abs(x_orig-x_next))
        plt.title('actual changes')
        
        plt.subplot(2, 3, 6)
        plt.axis("off")
        plt.imshow(np.abs(x_orig-x_pred))
        plt.title('predicted changes')

        plt.suptitle('epoch {}, index {}, vae_loss {:2.4f}, loss {:2.4f}'.format(epoch, i, loss_vae[i].cpu().data.item(), loss[i].cpu().data.item()))
#         plt.subplots_adjust(wspace=-.4, hspace=.1)#, bottom=0.1, right=0.8, top=0.9)
        plt.show()
        


In [145]:
import collections

def train(make_env, vae, mdnrnn, optimizer, test=False, cuda=True, batch_size=30, num_batches=100):
    try:
        env=make_env('sonic')
        vae.eval()
        if test:
            mdnrnn.eval()
        else:
            mdnrnn.train()
        info = collections.defaultdict(list)

        with tqdm(total=num_batches*batch_size, mininterval=0.5, desc='test' if test else 'training') as prog:
            for i in range(num_batches):
                # do a rollout
                actions = []
                observations = []
                rewards=[]

                observation = env.reset()
                for i in range(batch_size):
                    action = env.action_space.sample()
                    observation, reward, done, env_info = env.step(action)
                    actions.append(action)
                    observations.append(observation)
                    rewards.append(reward)
                    if done:
                        observation = env.reset()

                # stack
                actions = np.stack(actions)
                observations = np.stack(observations)
                rewards = np.stack(rewards)

                # Run VAE
                X = Variable(torch.from_numpy(observations)).cuda().transpose(1,3).contiguous()
                Y, mu, logvar = vae.forward(X)
                z = vae.sample(mu, logvar).data.cpu().numpy()
#                 vae_loss = loss_function(Y, X, mu, logvar).mean()

                # Stack into sequences
                # TODO stack as torch vars
                actions = timeseries_to_seq(actions, window=seq_len)
                z = timeseries_to_seq(z, window=seq_len)

                # Forward
                z_v = Variable(torch.from_numpy(z))
                actions_v = Variable(torch.from_numpy(actions.astype(np.uint8))).float()
                if cuda:
                    z_v=z_v.cuda()
                    actions_v=actions_v.cuda()
                z_pred, hidden_state = mdnrnn.forward(z_v, actions_v)

                loss = mdnrnn.rnn_loss(z_v, z_pred).mean()
                info['loss'].append(loss.cpu().data.numpy())

                if not test:
                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()

                prog.update(batch_size)
                prog.desc='loss={:2.4f}'.format(np.mean(info['loss']))

            print('[{}/{}] loss={:2.4f}'.format(i, num_batches*batch_size, np.mean(info['loss'])))
            prog.close()
            env.close()
    except:
        env.close()
        raise
    return info

In [154]:
# Load MDRNN
z_dim, action_dim, hidden_size, n_mixture, temp = 128, 12, 256, 5, 0.0
batch_size = 40
seq_len = 4

mdnrnn = MDNRNN(z_dim, action_dim, hidden_size, n_mixture, temp)

if cuda:
    mdnrnn = mdnrnn.cuda()

optimizer = optim.Adam(mdnrnn.parameters(), lr=1e-5)
import torch.optim.lr_scheduler
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=3, verbose=True)

In [ ]:
from custom_envs.env import make_env
current_env_name= 'sonic'
num_batches = 1000
num_epochs = 200

infos=[]
for epoch in range(num_epochs):
    info = train(make_env, vae, mdnrnn, optimizer, test=False, cuda=True, num_batches=num_batches, batch_size=batch_size)

    info_val = train(make_env, vae, mdnrnn, optimizer, test=True, cuda=True)
    scheduler.step(np.mean(info_val['loss']))
    
    print('Epoch {}, loss={:2.4f}, loss_val={:2.4f}'.format(epoch, np.mean(info['loss']), np.mean(info_val['loss'])))
    infos.append([info, info_val])
    
    plot_results(n=2, epoch=epoch)
    torch.save(vae.state_dict(), f'./models/{NAME}_{epoch}_state_dict.pkl')

/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/tqdm/_monitor.py:89: TqdmSynchronisationWarning: Set changed size during iteration (see https://github.com/tqdm/tqdm/issues/481)
  TqdmSynchronisationWarning)


# DEBUG can we replace tf_normal with torch normal?

In [ ]:
raise Exception('sd')

In [ ]:
# DEBUG run one train in global scope
from custom_envs.env import make_env
current_env_name= 'sonic'
batch_size = 20
num_batches = 1000
num_epochs = 20


env = make_env(current_env_name)


# do a rollout
actions = []
observations = []
rewards=[]

observation = env.reset()
for i in range(batch_size):
    action = env.action_space.sample()
    observation, reward, done, info = env.step(action)
    actions.append(action)
    observations.append(observation)
    rewards.append(reward)
    if done:
        observation = env.reset()

# stack
actions = np.stack(actions)
observations = np.stack(observations)
rewards = np.stack(rewards)
#         actions.shape

# Run VAE
X = Variable(torch.from_numpy(observations)).cuda().transpose(1,3).contiguous()
Y, mu, logvar = vae.forward(X)
z = vae.sample(mu, logvar).data.cpu().numpy()
#         z.shape

# Stack into sequences
actions = timeseries_to_seq(actions, window=seq_len)
z = timeseries_to_seq(z, window=seq_len)

# Forward
z_v = Variable(torch.from_numpy(z)).cuda()
actions_v = Variable(torch.from_numpy(actions.astype(np.uint8))).float().cuda()
if cuda:
    z_v=z_v.cuda()
    actions_v=actions_v.cuda()
z_pred, hidden_state = mdnrnn.forward(z_v, actions_v)

loss = mdnrnn.rnn_loss(z_v, z_pred)

optimizer.zero_grad()
loss.sum().backward()
optimizer.step()

env.close()

In [ ]:
# DEBUG compare loss with https://github.com/hardmaru/pytorch_notebooks/blob/master/mixture_density_networks.ipynb
oneDivSqrtTwoPI = 1.0 / np.sqrt(2.0*np.pi) # normalization factor for Gaussians
def gaussian_distribution(y, mu, sigma):
    # make |mu|=K copies of y, subtract mu, divide by sigma
    expand_by=mu.size(-1)//y.size(-1)
    y = y.repeat((1,1,expand_by))
#     result = (y.expand_as(mu) - mu) * torch.reciprocal(sigma)
    result = (y - mu) * 1/sigma
    result = -0.5 * (result * result)
    result = (torch.exp(result) /sigma) * oneDivSqrtTwoPI
    result = result.view((result.size(0), result.size(1), expand_by, -1))
    return result.sum(-1)

def mdn_loss_fn(pi, sigma, mu, y):
    result = gaussian_distribution(y, mu, sigma) 
    result = result * pi
    result = torch.sum(result, dim=1)
    result = -torch.log(result+1e-8)
    return torch.mean(result,1)

pi, mu, sigma = mdnrnn.get_mixture_coef(z_pred)
loss2=mdn_loss_fn(pi, sigma, mu, z_v)

loss1=mdnrnn.rnn_loss(z_v, z_pred)
print(loss1.sum(),loss2.sum())

In [ ]:
import torch.distributions
# replace tf_normal with pytorch normal dist logprob
rollout_length = z_v.size(1)
z_v2 = z_v.repeat((1,1,mdnrnn.n_mixture))
mu2 = mu.view((-1, rollout_length, mdnrnn.n_mixture, mdnrnn.z_dim))
sigma2 = sigma.view((-1, rollout_length, mdnrnn.n_mixture, mdnrnn.z_dim))
z_v2 = z_v2.view(-1, rollout_length, mdnrnn.n_mixture, mdnrnn.z_dim)
z_v2.shape, z_v.shape, mu.shape

z_normals = torch.distributions.Normal(mu2, sigma2)
z = z_normals.sample()
z_prob = z_normals.log_prob(z_v2).exp()
mu.shape, sigma.shape, z_prob.shape

In [ ]:
z_v.shape, z.shape

In [ ]:
# TODO confirm this works and the dimensions shapes are right
from torch import normal, multinomial

pi, mean, sigma = mdnrnn.get_mixture_coef(z_pred)

batch_size, seq_len, z_dim  = z_pred.size(0)
# randomly draw a mixture model

# pi = pi.contiguous().view(-1), 
# mean = mean.contiguous().view(-1), 
# sigma = sigma.contiguous().view(-1)

pi = pi.view(batch_size,-1)
mean = mean.contiguous().view(batch_size,-1)
sigma = sigma.contiguous().view(batch_size,-1)
k = multinomial(pi, 1).long() # one for each batch

# Multi d select
start = (mdnrnn.z_dim * k).squeeze()
end = (mdnrnn.z_dim * (k+1)).squeeze()
index = torch.stack([torch.arange(start[i],end[i], dtype=torch.long) for i in range(len(start))])
if cuda:
    index=index.cuda()

selected_mean = torch.stack([torch.index_select(mean[batch], index=index[batch], dim=0) for batch in range(batch_size)],0)

selected_sigma = torch.stack([torch.index_select(sigma[batch], index=index[batch], dim=0) for batch in range(batch_size)],0)
print(selected_mean.shape, selected_sigma.shape)

z = normal(selected_mean, selected_sigma)
z.shape